# Structure Property visualizer

This is the second of two deliverables for the SiSc-Lab2020 project.

Authors = 

Supervisors: Dr. Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

In [ ]:
# Instructions by supervisors
a = '''
## Jens
You have to implement this notebook

In the end only Markup text and output results should be seen if one hides the code cells (hide_code extension)

That is can easily exported into a nice pdf file.

Also the amount of python code in this notebook should be minimal.

Rather export the functions you use from a different file.

optional dump query results in a file, from which results will be reread for speed, i.e cache results

______________

An example of what we in the end aim for your find here.

https://www.materialscloud.org/discover/mofs#mcloudHeader

Clicking at one of the prerendered plots will open an interactive bokeh app.

Code for this you find here: https://github.com/materialscloud-org/structure-property-visualizer/blob/master/figure/main.py

For you sisc project this is to much, so do not view this as something you have to deliver but something that with the help of your work
in this project we will build out of it for our purposes.

So do not worry about an app.
_________________
**Your tasks are as follows:**

1. Implement a general interactive bokeh scatter plots with linked histograms (see static version examples/MP_convergence_scf_clean_150_240.png). 

   We gladly help you with it. (this goes into helpers.py and you import it here for usage). This function should not contain any aiida methods.
2. Extract float data you find in certain 'Dict' nodes into a/several pandas object, which will be then the datasource for this notebook. 

   So this notebook should not directly depend on any aiida methods (beyond load_profile)

## Johannes

Nothing much to add.

I'd just have bonus: if all this works, one could think about some actual data analysis, like clustering analysis or dimension reduction as we learned 
in Data Analysis & Visualization. For example, do [PCA](https://blog.exploratory.io/an-introduction-to-principal-component-analysis-pca-with-2018-world-soccer-players-data-810d84a14eab) 
with the quantities magnetic moment, energy, band gap, fermi energy, (structure, core levels). Ie, are some of these quantities (linearly) correlated in a 
data subset (e.g. all output nodes)? Since the data is already preprocessed mostly, this should be relatively easy, just throw the data into 
[scikit-learn](https://scikit-learn.org/stable/).
'''

In [ ]:
# Imports
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook

In [ ]:
# python imports:
from collections import Counter
import time
import numpy as np
import pandas as pd
#from pprint import pprint

# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add further imports
# project imports:
from aiida_jutools.sisc_lab import helpers
#import helpers
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

# Subastk D2.a: Data acquisition

Task: check which output dict nodes returned by workflows, which had `StructureData` nodes as inputs are there in the database.

For example, for a successful `FleurScfWorkChain`, there are two return `Dict` nodes, one is linked with
`last_fleur_calc_output` and one with `output_scf_wc_para`.

If a `StructureData` is an input of such a workflow you can extract the formula, `uuid` and other information you need from the 
`StructureData` which is always linked into workflows via the link name `structure`.

All the user should have to say:
```python
source = generate_structure_property_pandas_source('<workflow_name>')
```


In [ ]:
#Find out valid plugin tools
# !verdi plugin list aiida.workflows

#Have a look at the attributes of dict nodes and sturcture nodes
# dict_project=['attributes','uuid']
# workflowdictlst = helpers.get_structure_workflow_dict(dict_project=dict_project)
#or
# structure_project=['uuid', 'extras','attributes']
# workflowdictlst = helpers.get_structure_workflow_dict(structure_project=structure_project)

In [ ]:
# workflow_name = 'fleur_scf_wc' # Filter workflow
# workflowdictlst = helpers.get_structure_workflow_dict(workflow_filters={'attributes.process_label':workflow_name})
#or
workflow_name = None # No restriction. Querying by default
workflowdictlst = helpers.get_structure_workflow_dict()

print(len(workflowdictlst))
workflowdictlst[:2]

In [ ]:
# Some dict attributes: force, energy, total_energy, fermi_energy, bandgap, charge_density, distance_charge, 
#                       workflow_version, parser_info, material,
# Remarks:  workflow_version=0.2.2: energy, force
#           workflow_version=0.4.2: total_energy, material. 
#           parser: fermi_energy, bandgap.

dict_project=['uuid', 'attributes.force', 'attributes.energy', 'attributes.total_energy', 'attributes.distance_charge',             'attributes.bandgap', 'attributes.workflow_version', 'attributes.parser_info', 'attributes.material',
        'attributes.fermi_energy']
dictpd = helpers.generate_dict_property_pandas_source(
            workflow_name, 
            dict_project=dict_project, 
            filename='dict_property.json')
dictpd.head(5)

In [ ]:
structure_project=['uuid', 'extras.formula']
structurepd = helpers.generate_structure_property_pandas_source(
            workflow_name, 
            structure_project=structure_project,
            filename='structure_property.json')
structurepd.head(5)

In [ ]:
# Generate the combination of the above
combinepd = helpers.generate_combination_property_pandas_source(
            workflow_name, 
            dict_project=dict_project, 
            structure_project=structure_project,
            filename='combined_property.json')
combinepd.head(5)

# Subtask D2.b: Interactive plot

Allow the user to choose, which properties to plot on what axis.

```python
xdata = source['distance']
ydata = source['energy']
```

Single bokeh scatter plot with histpgrams on both sides, hover tool should show 'input structure, formula, 
structure_uuid and dictnode uuid', as long as this information is available.

```python
bokeh_struc_prop_vis(xdata, ydata, src=source, **kwargs)
```

In [ ]:
from helpers import read_json_file
df, xdata, ydata = read_json_file('combined_property.json','energy', 'bandgap')
df

In [ ]:
from helpers import bokeh_struc_prop_vis
bokeh_struc_prop_vis('combined_property.json', 'energy', 'bandgap')